# «به نام خدا»
## پردازش زبان طبیعی 
## تمرین کامپیوتری شماره 3 - بخش دوم
## تشخیص گروه هاي اسمی
### امیرحسین دبیری اقدم - 810197502
#####  دانشگاه تهران - اردیبهشت 1401

# **Named Entity Recognition**

In [1]:
!pip install seqeval

In [2]:
import numpy as np
import nltk
from nltk import conlltags2tree, tree2conlltags
from seqeval.metrics import precision_score, f1_score, recall_score
# from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import treebank
nltk.download('treebank')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
tagged_sents = list()
for sent in treebank.tagged_sents():
    tagged_sents.append(tree2conlltags(nltk.ne_chunk(sent)))

for i in range(len(tagged_sents)):
    for j in range(len(tagged_sents[i])):
        tagged_sents[i][j] = (tagged_sents[i][j][0], tagged_sents[i][j][2])

In [4]:
print(tagged_sents[13])

[('The', 'O'), ('Lorillard', 'B-ORGANIZATION'), ('spokeswoman', 'O'), ('said', 'O'), ('0', 'O'), ('asbestos', 'O'), ('was', 'O'), ('used', 'O'), ('*-1', 'O'), ('in', 'O'), ('``', 'O'), ('very', 'O'), ('modest', 'O'), ('amounts', 'O'), ("''", 'O'), ('in', 'O'), ('*', 'O'), ('making', 'O'), ('paper', 'O'), ('for', 'O'), ('the', 'O'), ('filters', 'O'), ('in', 'O'), ('the', 'O'), ('early', 'O'), ('1950s', 'O'), ('and', 'O'), ('replaced', 'O'), ('*-1', 'O'), ('with', 'O'), ('a', 'O'), ('different', 'O'), ('type', 'O'), ('of', 'O'), ('filter', 'O'), ('in', 'O'), ('1956', 'O'), ('.', 'O')]


In [5]:
freq_dict = dict()
for sent in tagged_sents:
    for word in sent:
        tag = word[1]
        if (tag in freq_dict): freq_dict[tag] += 1
        else: freq_dict[tag] = 1
        
print(freq_dict)

{'B-PERSON': 2086, 'B-ORGANIZATION': 1567, 'O': 92442, 'B-GPE': 1895, 'I-ORGANIZATION': 878, 'I-PERSON': 1326, 'B-LOCATION': 34, 'I-LOCATION': 32, 'B-FACILITY': 53, 'I-FACILITY': 56, 'I-GPE': 265, 'B-GSP': 40, 'I-GSP': 2}


In [6]:
np.random.seed(810197502)
np.random.shuffle(tagged_sents)

In [7]:
train_sents = tagged_sents[978:] # 75%
test_sents = tagged_sents[:978] # 25%

In [8]:
S = {0:'O', 1:'B-PERSON', 2:'I-PERSON', 3:'B-ORGANIZATION', 4:'I-ORGANIZATION', 5:'B-GPE', 6:'I-GPE', 7:'B-LOCATION', 8:'I-LOCATION', 9:'B-FACILITY', 10:'I-FACILITY', 11:'B-GSP', 12:'I-GSP'}

In [9]:
A = list()

#START
temp = dict()
count = 0
for sent in train_sents:
        count += 1
        if (sent[0][1] in temp): temp[sent[0][1]] += 1
        else: temp[sent[0][1]] = 1
for k, v in temp.items():
    if(count > 0): temp[k] = temp[k] / count
A.append(temp)

for s in S:
    temp = dict()
    count = 0
    for sent in train_sents:
        for i in range(len(sent) - 1):
            word_i = sent[i]
            word_i1 = sent[i + 1]
            if (word_i[1] == S[s]):
                count += 1
                if (word_i1[1] in temp): temp[word_i1[1]] += 1
                else: temp[word_i1[1]] = 1
        if (sent[-1][1] == S[s]):
            count += 1
            if ('END' in temp): temp['END'] += 1
            else: temp['END'] = 1
    for k, v in temp.items():
        if (count > 0): temp[k] = temp[k] / count
    A.append(temp)  

In [10]:
A

[{'B-FACILITY': 0.0006811989100817438,
  'B-GPE': 0.09196185286103542,
  'B-GSP': 0.0010217983651226157,
  'B-ORGANIZATION': 0.021798365122615803,
  'B-PERSON': 0.11171662125340599,
  'O': 0.7728201634877384},
 {'B-FACILITY': 0.0006153934224461173,
  'B-GPE': 0.016257835532529984,
  'B-GSP': 0.00030054097375275494,
  'B-LOCATION': 0.0003434753985745771,
  'B-ORGANIZATION': 0.014282851990726163,
  'B-PERSON': 0.016472507656639095,
  'END': 0.041946933050920225,
  'O': 0.9097804619744111},
 {'B-GPE': 0.014303482587064677,
  'B-ORGANIZATION': 0.046019900497512436,
  'B-PERSON': 0.06529850746268656,
  'END': 0.0006218905472636816,
  'I-PERSON': 0.5261194029850746,
  'O': 0.347636815920398},
 {'B-PERSON': 0.002944062806673209,
  'I-PERSON': 0.16977428851815504,
  'O': 0.8272816486751717},
 {'B-GPE': 0.0008403361344537816,
  'B-ORGANIZATION': 0.02100840336134454,
  'B-PERSON': 0.0058823529411764705,
  'END': 0.0008403361344537816,
  'I-ORGANIZATION': 0.4084033613445378,
  'O': 0.563025210084

In [11]:
B = list()
for s in S:
    temp = dict()
    count = 0
    for sent in train_sents:
        for word in sent:
            if (word[1] == S[s]):
                count += 1
                if (word[0] in temp): temp[word[0]] += 1
                else: temp[word[0]] = 1
    for k, v in temp.items():
        if(count > 0): temp[k] = temp[k] / count
    B.append(temp)    

In [12]:
B

[{'has': 0.0036494261098548816,
  'written': 2.862294988121476e-05,
  'to': 0.023427884477774278,
  'the': 0.04427970346623923,
  'chairman': 0.00047227867304004353,
  'of': 0.024672982797607122,
  'Olympic': 4.2934424821822136e-05,
  'Committee': 2.862294988121476e-05,
  '*-2': 0.004035835933251281,
  'asking': 8.586884964364427e-05,
  'him': 0.00018604917422789593,
  '*-3': 0.0014454589690013453,
  'back': 0.00024329507399032543,
  'a': 0.020078999341672153,
  'bid': 0.00035778687351518447,
  '*': 0.01021839310759367,
  'join': 4.2934424821822136e-05,
  'committee': 0.0001431147494060738,
  ',': 0.05288089990554427,
  'news': 0.0003148524486933623,
  'agency': 0.00015742622434668116,
  'said': 0.007112803045481867,
  '0': 0.01206457337493202,
  '*T*-1': 0.008916048887998396,
  '.': 0.04114549045424622,
  'The': 0.007585081718521911,
  'new': 0.0017889343675759224,
  'company': 0.0030340326874087643,
  'will': 0.0031199015370524086,
  'attempt': 0.00010018032458425165,
  '*-1': 0.0123

In [13]:
def VITERBI(O, S, A, B, not_seen_prob = 1e-6):
    viterbi = np.zeros([len(S) + 2, len(O)])
    backpointer = np.zeros([len(S) + 2, len(O)])
    for s in S:
        viterbi[s][0] = (A[0][S[s]] if (S[s] in A[0]) else not_seen_prob) * (B[s][O[0]] if (O[0] in B[s]) else not_seen_prob)
        backpointer[s][0] = 0
    for t in range(1, len(O)):
        for s in S:
            viterbi[s][t] = np.max([viterbi[i][t-1] * (A[i][S[s]] if (S[s] in A[i]) else not_seen_prob) * (B[s][O[t]] if (O[t] in B[s]) else not_seen_prob) for i in S])
            backpointer[s][t] = np.argmax([viterbi[i][t-1] * (A[i][S[s]] if (S[s] in A[i]) else not_seen_prob) for i in S])
    
    viterbi[len(S) + 1][len(O) - 1] = np.max([viterbi[s][len(O) - 1] * (A[s]['END'] if ('END' in A[s]) else not_seen_prob) for s in S])
    backpointer[len(S) + 1][len(O) - 1] = np.argmax([viterbi[s][len(O) - 1] * (A[s]['END'] if ('END' in A[s]) else not_seen_prob) for s in S])


    best_path = list()
    k = np.argmax([viterbi[s][len(O)-1] for s in S]) 
    for i in range(len(O)-1, -1, -1):  
        best_path.insert(0, (O[i], S[k]))         
        k = int(backpointer[k][i])                
    return best_path

In [14]:
def pre_process(sent):
    temp = list()
    temp2 = list()
    for word in sent:
            temp.append(word[0])
            temp2.append(word[1])
    return temp, temp2

In [15]:
def compute_precision_f1_recall(test_sents):
    correct = 0
    total = 0
    y_test = list()
    for sent in test_sents:
        temp = list()
        for word in sent:
            temp.append(word[1])
        y_test.append(temp)
    
    y_pred = list()
    not_seen_prob = 1 / len(treebank.tagged_words())
    for sent in test_sents:
        sent, true_tags = pre_process(sent)
        pred_tags = VITERBI(sent, S, A, B, not_seen_prob)
        temp = list()
        for tag in pred_tags:
            temp.append(tag[1])
        y_pred.append(temp)

    return precision_score(y_test, y_pred), f1_score(y_test, y_pred), recall_score(y_test, y_pred)


In [19]:
compute_precision_f1_recall(test_sents)

(0.38156996587030717, 0.397158081705151, 0.4140740740740741)